In [8]:
import plotly.graph_objs as go
from plotly.offline import iplot
from sklearn.preprocessing import PowerTransformer
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
df = pd.read_csv('data/GDPC1.csv')

In [10]:
df = df.set_index(['DATE'])
df.index = pd.to_datetime(df.index)
if not df.index.is_monotonic:
    df = df.sort_index()

df = df.rename(columns={'GDPC1_PCH': 'value'})

In [11]:
def generate_time_lags(df, n_lags):
    df_n = df.copy()
    for n in range(1, n_lags + 1):
        df_n[f"lag{n}"] = df_n["value"].shift(n)
    df_n = df_n.iloc[n_lags:]
    return df_n

In [12]:
input_dim = 16
df_lags = generate_time_lags(df, input_dim)

In [14]:
from sklearn.model_selection import train_test_split
def feature_label_split(df, target_col):
    y = df[[target_col]]
    X = df.drop(columns=[target_col])
    return X, y

def train_val_test_split(df, target_col, test_ratio):
    val_ratio = test_ratio / (1 - test_ratio)
    X, y = feature_label_split(df, target_col)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio, shuffle=False)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [15]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(df_lags, 'value', 0.2)

scaler = PowerTransformer()
X_train_arr = scaler.fit_transform(X_train)
X_val_arr = scaler.transform(X_val)
X_test_arr = scaler.transform(X_test)

y_train_arr = scaler.fit_transform(y_train)
y_val_arr = scaler.transform(y_val)
y_test_arr = scaler.transform(y_test)

batch_size = 64

train_features = torch.Tensor(X_train_arr)
train_targets = torch.Tensor(y_train_arr)
val_features = torch.Tensor(X_val_arr)
val_targets = torch.Tensor(y_val_arr)
test_features = torch.Tensor(X_test_arr)
test_targets = torch.Tensor(y_test_arr)

train = TensorDataset(train_features, train_targets)
val = TensorDataset(val_features, val_targets)
test = TensorDataset(test_features, test_targets)

train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader_one = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)

In [20]:
data = iter(train_loader) # Let's iterate on it
single_point = next(data)
print(f"""Type: {type(single_point)}
Length: {len(single_point)}
More Types: {type(single_point[0])}, {type(single_point[1])}
Shapes: {single_point[0].shape}, {single_point[1].shape}
Labels: {single_point[1]}
""")

Type: <class 'list'>
Length: 2
More Types: <class 'torch.Tensor'>, <class 'torch.Tensor'>
Shapes: torch.Size([64, 16]), torch.Size([64, 1])
Labels: tensor([[ 1.8615e-01],
        [-6.5562e-01],
        [-1.6516e-01],
        [ 2.6378e+00],
        [ 1.0245e+00],
        [-1.1411e-01],
        [-1.3430e+00],
        [-2.1048e+00],
        [-1.2718e+00],
        [-7.5459e-01],
        [ 2.4892e-01],
        [ 1.1314e+00],
        [ 2.1417e+00],
        [ 7.7393e-01],
        [ 4.7969e-01],
        [-2.8550e-01],
        [-1.1955e+00],
        [-6.0495e-02],
        [-9.3499e-01],
        [ 7.9397e-01],
        [-2.4610e-01],
        [-1.0492e+00],
        [ 9.5399e-02],
        [-1.7286e+00],
        [-2.9107e+00],
        [-2.2934e-01],
        [ 1.5262e+00],
        [ 1.5540e+00],
        [ 1.0907e+00],
        [ 1.4613e+00],
        [-7.8993e-01],
        [-5.9016e-01],
        [ 1.4530e+00],
        [-1.3232e+00],
        [-3.9395e-01],
        [-1.9257e+00],
        [-2.1184e-01],
 

In [16]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(RNNModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # RNN layers
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, h0 = self.rnn(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out

In [19]:
from torch import optim
from tqdm import tqdm # This is optional but useful

# Let's get the right torch device (preference of GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Let's set up some parameters
input_dim = len(X_train.columns)
print(input_dim)
output_dim = 1
hidden_dim = 64
layer_dim = 3
batch_size = 64
dropout = 0.2
n_epochs = 100
learning_rate = 1e-3
weight_decay = 1e-6

model = RNNModel(input_dim=input_dim,
                 hidden_dim=hidden_dim,
                 layer_dim=layer_dim,
                 output_dim=output_dim,
                 dropout_prob=dropout).to(device)
print(model)
# We need an optimizer that tells us what form of gradient descent to do
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# We also need a loss function
LossFunction = nn.CrossEntropyLoss()

# This is default on but let's just be pedantic
model.train()
loss_history = []
loss = torch.Tensor([0])
for epoch in tqdm(range(n_epochs),
                  desc=f"Epoch",
                  unit="epoch",
                  disable=False):
    for (data, label) in tqdm(train_loader,
                              desc="iteration",
                              unit="%",
                              disable=True):
        optimizer.zero_grad(set_to_none=True) # Here we clear the gradients
        
        # We need to make sure the tensors are on the same device as our model
        data = data.to(device)
        label = label.to(device)
        out = model(data)
        
        loss = LossFunction(out, label)
        
        # PyTorch is Magic!
        loss.backward() # This function calculates all our gradients
        optimizer.step() # This function does our gradient descent with those gradients
        loss_history.append(loss.item())
    print(f"Epoch {epoch}: loss: {loss.item()}")


16
RNNModel(
  (rnn): RNN(16, 64, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]


RuntimeError: input must have 3 dimensions, got 2